# 获取源文件配置路径
将所有需要进行分词的文件的根目录放置在json配置文件中

获取所有根目录，根据里面的工作站文件，深搜地进行分词

将所有失败的文件路径，保存到一个失败重试的文件目录中

In [1]:
import json

text_split_path = "D:\\workSpace\\Scripts\\python_scripts\\text_split\\config\\textSplit.json"
with open(text_split_path,'r', encoding='utf-8') as f:
    config = json.load(f)
    
fp = config['file_path']
failed_root = config['failed_path']

## 深搜分词
根据工作站文件递归分词

In [12]:
from typing import List
import os
from docx import Document
from textSplitter.chinese_recursive_text_splitter import ChineseRecursiveTextSplitter

def read_docx(docx_path:str)->str:
    # 初始化一个Document对象，读取docx文件
    doc = Document(docx_path)

    # 初始化一个空字符串来存储文档中的纯文本
    full_text = []

    # 遍历文档中的每个段落
    for para in doc.paragraphs:
        # 添加段落文本到full_text列表中
        full_text.append(para.text)

        # 将列表中的文本合并成一个字符串并返回
    return '\n'.join(full_text)


def do_fail_rec(task):
    with open(failed_root,'a+',encoding='utf-8') as file:
        file.write(task+'\n')


def do_split(task:str)->List[str]:
    try:
        text = read_docx(task)
        # 调分词器
        t_splitter = ChineseRecursiveTextSplitter(
            keep_separator=True,
            is_separator_regex=True,
            chunk_size=50,
            chunk_overlap=0
        )
        
        res = t_splitter.split_text(text)
        return res
    except Exception as e:
        print(f'对{task}进行分词时出现错误:{e}')
        do_fail_rec(task)
    

def do_save(target,res,task):
    global i
    try:
        for i in range(len(res)):
            target = os.path.join(target,task+f'_split{i}.txt')
            with open(target,'w',encoding='utf-8') as tar:
                tar.write(res[i])
    except Exception as e:
        print(f'保存{task}_{i}时出现错误:{e}')
    

def dfs_split(path:str, work_station:str, target:str):
    # 获取工作站名称
    ws = os.sep.join([path,work_station])
    # 副本
    ws_tmp = work_station.replace('.txt','.tmp')
    ws_tmp = os.sep.join([path,ws_tmp])
    # print(f'{ws}\n{ws_tmp}')
    
    dfs = True
    while dfs:
        with open(ws,'r+',encoding='utf-8') as file, open(ws_tmp, 'a+',encoding='utf-8') as tmp:
            lines = file.readlines()
            
            if len(lines) == 0:
                dfs = False
            
            # 读取第一条任务
            task = path+'\\'+lines[0].rstrip()
            # 剩余任务写入副本
            for i in range(1,len(lines)):
                tmp.write(lines[i].rstrip()+'\n')
            print(f'正在处理任务:{task}')
            
            # 文件夹深搜
            if os.path.isdir(task):
                dfs_split(task,work_station,target)
            else:
                # 分词
                res = do_split(task)
                # 保存
                do_save(target,res,task)
    
        if os.path.exists(ws):
            os.remove(ws)
        # 重命名副本覆盖工作站
        os.rename(ws_tmp,ws)       

正在处理任务:D:\workSpace\Scripts\python_scripts\text_split\source_files\《周易研究》2020-2023\2020
正在处理任务:D:\workSpace\Scripts\python_scripts\text_split\source_files\《周易研究》2020-2023\2020\202001
正在处理任务:D:\workSpace\Scripts\python_scripts\text_split\source_files\《周易研究》2020-2023\2020\202001\“心为太极”说在朱子学中的诠释_焦德明.docx


KeyboardInterrupt: 

对所有source file的文件根目录进行深搜、分词、保存

In [4]:
for root in fp.values():
    work_station = root['WorkStation']
    src_paths = root['src_paths']
    target_paths = root['target_paths']
    for i in range(len(src_paths)):
        dfs_split(src_paths[i], work_station, target_paths[i])

KeyboardInterrupt: 